In [8]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import KFold
import os
from matplotlib import pyplot
%matplotlib inline
from numpy import mean
from numpy import std


In [9]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# load train and test dataset
def load_dataset():
    # load dataset
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    
    trainX = trainX.astype('float32')
    testX = testX.astype('float32')
    
    print(trainX.shape)
    # reshape dataset to have a single channel
    trainX = trainX.reshape((trainX.shape[0], 32, 32, 3))
    testX = testX.reshape((testX.shape[0], 32, 32, 3))
    # one hot encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
   
    return trainX, trainY, testX, testY


In [10]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
   
	return train_norm, test_norm

# define cnn model
def define_model(train_ix):
    model = Sequential()
    #Conv first pass
    model.add(Conv2D(32,
                     (3, 3),
                     padding='same',
                     activation='relu',
                     kernel_initializer='he_uniform',
                     input_shape = (32, 32, 3)))
   
    #Conv second pass
    model.add(Conv2D(32,
                     (3, 3),
                     padding='same',
                     activation='relu',
                     kernel_initializer='he_uniform',
                     ))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
      #Conv third pass
    model.add(Conv2D(64,
                     (3, 3),
                     padding='same',
                     activation='relu',
                     kernel_initializer='he_uniform',
                     ))
 
   
    #Conv fourth pass
    model.add(Conv2D(64,
                     (3, 3),
                     padding='same',
                     activation='relu',
                     kernel_initializer='he_uniform',
                     ))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    model.add(Dense(512, activation='relu',
                    kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    
    model.add(Dense(num_classes, activation='softmax'))
   
    # compile model
    opt = SGD(lr=0.01, momentum=0.9)
    
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    # printing the model summary
    print(model.summary())
    
    return model


In [11]:
# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds=5):
	scores, histories = list(), list()
	# prepare cross validation
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# enumerate splits
	for train_ix, test_ix in kfold.split(dataX):
		# define model
		model = define_model(dataX)
      
		# select rows for train and test
		trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		# fit model
		history = model.fit(trainX, trainY, epochs=epochs, 
                      batch_size=batch_size, validation_data=(testX, testY), 
                      verbose=0)
      
        
		# evaluate model
		loss, acc = model.evaluate(testX, testY, verbose=0)
		print('\n Accuracy: %.3f Loss: %.3f ' % (acc * 100.0))
		# append score
		scores.append(acc)
        
		histories.append(history)
	return scores, histories


In [12]:
# plot diagnostic learning curves
def summarize_diagnostics(histories):
	for i in range(len(histories)):
		# plot loss
		pyplot.subplot(211)
		pyplot.title('Cross Entropy Loss')
		pyplot.plot(histories[i].history['loss'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_loss'], 
              color='orange', label='test')
		# plot accuracy
		pyplot.subplot(212)
		pyplot.title('Classification Accuracy')
		pyplot.plot(histories[i].history['acc'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_acc'], 
              color='orange', label='test')
	pyplot.show()

In [13]:
# summarize model performance
def summarize_performance(scores):
	# print summary
	print('Accuracy: mean=%.3f std=%.3f, n=%d' 
       % (mean(scores) *100, std(scores)*100, len(scores)))
	# box and whisker plots of results
	pyplot.boxplot(scores)
	pyplot.show()
    

# run the test harness for evaluating a model
def run_test_harness():
    # load dataset
    trainX, trainY, testX, testY = load_dataset()
    
    # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)
  
    # evaluate model  
    scores, histories = evaluate_model(trainX, trainY)
    
    print(histories)
    # learning curves
    summarize_diagnostics(histories)
    # summarize estimated performance
    summarize_performance(scores)


In [14]:
run_test_harness

<function __main__.run_test_harness()>